In [ ]:
%pip install numpy pandas 

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
cust_df=pd.read_csv('customers.csv')
trans_df=pd.read_csv('transactions.csv')
prod_df=pd.read_csv('products.csv')

Part A: Data Loading & Basic Operations 



In [ ]:
print("Shape of Transactions:", trans_df.shape)
print("Shape of Customers:", cust_df.shape)
print("Shape of Products:", prod_df.shape)

In [ ]:
print("Data type of each column in Transactions:\n", trans_df.dtypes)
print("Data type of each column in Customers:\n", cust_df.dtypes)
print("Data type of each column in Products:\n", prod_df.dtypes)

In [ ]:
print("FIrst 3 rows of Transactions:\n", trans_df.head(3))
print("FIrst 3 rows of Customers:\n", cust_df.head(3))
print("FIrst 3 rows of Products:\n", prod_df.head(3))

In [ ]:
print("Missing values in Transactions:\n", trans_df.isnull().sum())
print("Missing values in Customers:\n", cust_df.isnull().sum())
print("Missing values in Products:\n", prod_df.isnull().sum())

In [ ]:
trans_duploicated=trans_df.duplicated().sum()
cust_duploicated=cust_df.duplicated().sum()
prod_duploicated=prod_df.duplicated().sum()
print("Duplicate rows in Transactions:", trans_duploicated)
print("Duplicate rows in Customers:", cust_duploicated)
print("Duplicate rows in Products:", prod_duploicated)

In [ ]:
trans_df['timestamp'] = pd.to_datetime(trans_df['timestamp'])

earliest_date = trans_df['timestamp'].min()
latest_date = trans_df['timestamp'].max()

print("Earliest transaction date:", earliest_date)
print("Latest transaction date:", latest_date)

In [ ]:
trans_df['hours']=trans_df['timestamp'].dt.hour
trans_df['day_of_week']=trans_df['timestamp'].dt.day_name()
trans_df['months']=trans_df['timestamp'].dt.month_name()

In [ ]:
trans_df.head()
# trans_df.drop(['days'], inplace=True, axis=1)

Part B: Data Cleaning & Transformation 

In [ ]:
num_cols = trans_df.select_dtypes(include=['number']).columns
trans_df[num_cols] = trans_df[num_cols].fillna(trans_df[num_cols].median())

cat_cols = trans_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    trans_df[col] = trans_df[col].fillna(trans_df[col].mode()[0])


# print("Missing values after handling:\n", trans_df.isnull().sum())

In [ ]:
trans_df['revenue'] = trans_df['quantity'] * trans_df['price']
trans_df.head()

In [ ]:
full_data=trans_df.merge(cust_df, on='customer_id', how='left').merge(prod_df, on='product_id', how='left')
full_data.head()

In [ ]:
# full_data.columns

In [ ]:
full_data['profit_margin']=(full_data['price']-full_data['cost_price'])/full_data['price']*100
full_data.head()

Part C: Aggregation & Analysis 

C1. Customer Metrics

In [ ]:
# 1. Total revenue generated, number of transactions, average transaction value
customer_summary = (
    full_data.groupby('customer_id').agg
    (
        total_revenue=('revenue', 'sum'),
        num_transactions=('transaction_id', 'count'),
        avg_transaction_value=('revenue', 'mean'),
        customer_name=('customer_name', 'first'),
        country=('country', 'first')
    )
    .reset_index()
)

In [ ]:
# 2. Most frequently purchased category per customer
most_freq_category = (
    full_data.groupby(['customer_id', 'category']).size().reset_index(name='count')
    .sort_values(['customer_id', 'count'], ascending=[True, False])
    .drop_duplicates('customer_id')[['customer_id', 'category']]
    .rename(columns={'category': 'most_freq_category'})
)

In [ ]:
customer_summary = customer_summary.merge(most_freq_category, on='customer_id', how='left')
customer_summary = customer_summary.sort_values(by='total_revenue', ascending=False)

print(customer_summary.head())

C2. Time-Based Analysis

In [ ]:
# Group by 'months' to compute monthly metrics
month_summary = (
    full_data.groupby('months').agg(
        total_revenue=('revenue', 'sum'),
        unique_customers=('customer_id', 'nunique'),
        avg_order_value=('revenue', 'mean')
    )
    .reset_index()
)

In [ ]:
month_order = ['January','February','March','April','May','June','July','August','September','October','November','December']
month_summary['months'] = pd.Categorical(month_summary['months'], categories=month_order, ordered=True)
month_summary = month_summary.sort_values('months')

In [ ]:
month_summary['mom_growth_%'] = month_summary['total_revenue'].pct_change() * 100

month_summary

C3. Product Performance

In [ ]:
product_summary = (
    full_data.groupby(['product_id', 'product_name']).agg(
        total_revenue=('revenue', 'sum'),
        total_quantity=('quantity', 'sum'),
        total_cost=('cost_price', 'sum')
    )
    .reset_index()
)

In [ ]:
product_summary['profit'] = product_summary['total_revenue'] - product_summary['total_cost']
product_summary['profit_margin'] = (product_summary['profit'] / product_summary['total_revenue'].replace(0, pd.NA)) * 100

In [ ]:
# Pick top 10 properly
top_revenue = product_summary.nlargest(10, 'total_revenue')
top_quantity = product_summary.nlargest(10, 'total_quantity')
top_margin = product_summary.nlargest(10, 'profit_margin')

In [ ]:
print("\n Top 10 Products by Total Revenue:")
print(top_revenue[['product_name', 'total_revenue', 'total_quantity', 'profit_margin']].to_string(index=False))


In [ ]:
print("\n Top 10 Products by Total Quantity Sold:")
print(top_quantity[['product_name', 'total_quantity', 'total_revenue', 'profit_margin']].to_string(index=False))



In [ ]:
print("\n Top 10 Products by Profit Margin:")
print(top_margin[['product_name', 'profit_margin', 'total_revenue', 'total_quantity']].to_string(index=False))

C4. Customer Segmentation (RFM)


In [65]:
latest_date = full_data['timestamp'].max()

In [66]:
rfm = full_data.groupby('customer_id').agg({
    'timestamp': lambda x: (latest_date - x.max()).days,  # Recency
    'transaction_id': 'count',                            # Frequency
    'revenue': 'sum'                                      # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']


In [67]:
rfm['Recency_bin'] = pd.qcut(rfm['Recency'], q=3, labels=['High', 'Medium', 'Low'])
rfm['Frequency_bin'] = pd.qcut(rfm['Frequency'], q=3, labels=['Low', 'Medium', 'High'])
rfm['Monetary_bin'] = pd.qcut(rfm['Monetary'], q=3, labels=['Low', 'Medium', 'High'])


In [ ]:
rfm['Segment'] = (
    rfm['Recency_bin'].astype(str) + '-' +
    rfm['Frequency_bin'].astype(str) + '-' +
    rfm['Monetary_bin'].astype(str)
)

print("🔹 Sample of RFM Segmentation:")
print(rfm.head(10)[['customer_id', 'Recency', 'Frequency', 'Monetary',
                     'Recency_bin', 'Frequency_bin', 'Monetary_bin', 'Segment']])

🔹 Sample of RFM Segmentation:
   customer_id  Recency  Frequency   Monetary Recency_bin Frequency_bin  \
0            1      320          5   1424.990         Low        Medium   
1            2       38          3    641.990        High           Low   
2            3       38         30  11180.420        High          High   
3            4      172          8   4383.630      Medium          High   
4            5      143          5   1920.980      Medium        Medium   
5            6       12          6   2640.970        High        Medium   
6            7      189          3   1415.490         Low           Low   
7            8      251          3    941.250         Low           Low   
8            9      243          4   3080.180         Low        Medium   
9           10        2         22   9368.612        High          High   

  Monetary_bin               Segment  
0       Medium     Low-Medium-Medium  
1          Low          High-Low-Low  
2         High        High-